In [87]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
import time
import collections
import re

db = pd.read_csv("results.csv")

documents = db['text']
stopwords = stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agnal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\agnal\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Questão 01) Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [88]:
toker = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+')
inverted_list = {}
n = 0

for document in documents:
    n += 1
    tokens = toker.tokenize(document.lower())
    for token in set(tokens):
        if (token not in stopwords and len(token) >= 2):
            count = (n, tokens.count(token))
            if (token not in inverted_list.keys()):
                  inverted_list[token] = []
                  inverted_list[token].append(count)
            else:
                  inverted_list[token].append(count)
                
                
inverted_index_df = pd.DataFrame.from_dict(inverted_list, orient="index")
inverted_index_df.to_csv('data.csv')
inverted_index = inverted_list

2. Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

In [89]:
def document_at_time(query, inverted_index, k):
    inverted_lists = []

    r = []
    for word in query.split(" "):
        if word in inverted_index.keys():
            inverted_lists.append(inverted_index[word])
    for document in range(1, len(documents)+1):
        sd = 0
        for inverted_list in inverted_lists:
            for i in inverted_list:
                if (i[0] == document):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, document))
    return heapq.nlargest(k, r)
document_at_time("política", inverted_index, 10)

[(9, 69),
 (7, 235),
 (7, 166),
 (6, 204),
 (5, 173),
 (5, 165),
 (5, 152),
 (5, 138),
 (5, 63),
 (4, 221)]

Term a time


In [90]:
def term_at_time(query, inverted_index, k):
    
    a = {}
    inverted_lists = []
    r = []
    for word in query.split(" "):
        if word in inverted_index.keys():
            inverted_lists.append(inverted_index[word])
    for inverted_list in inverted_lists:
        for post in inverted_list:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))
    return heapq.nlargest(k, r)
term_at_time("política", inverted_index, 10)

[(9, 69),
 (7, 235),
 (7, 166),
 (6, 204),
 (5, 173),
 (5, 165),
 (5, 152),
 (5, 138),
 (5, 63),
 (4, 221)]

Defina 5 consultas de um termo somente.

In [91]:
queries = ["política", "presidente", "ministério", "brasil", "economia"]


Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo)

In [92]:
results_document = []
results_term = []
time_results_document = []
time_results_term = []
k = 10

for query in queries:
  init_document = time.time()
  results_document.append(document_at_time(query, inverted_index, k))
  end_document = time.time()
  time_results_document.append(end_document - init_document)
  
  init_term = time.time()
  results_term.append(term_at_time(query, inverted_index, k))
  end_term = time.time()
  time_results_term.append(end_term - init_term)

queries_df = pd.DataFrame()
queries_df['query'] = queries
queries_df['document_at_a_time'] = results_document
queries_df['term_at_a_time'] = results_term
queries_df['compare'] = queries_df.document_at_a_time == queries_df.term_at_a_time
queries_df

,query,document_at_a_time,term_at_a_time,compare
0,política,"[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...","[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...",True
1,presidente,"[(16, 166), (15, 63), (12, 151), (11, 216), (1...","[(16, 166), (15, 63), (12, 151), (11, 216), (1...",True
2,ministério,"[(8, 221), (7, 124), (7, 115), (4, 203), (3, 2...","[(8, 221), (7, 124), (7, 115), (4, 203), (3, 2...",True
3,brasil,"[(47, 151), (15, 166), (11, 19), (10, 26), (9,...","[(47, 151), (15, 166), (11, 19), (10, 26), (9,...",True
4,economia,"[(10, 138), (8, 125), (6, 127), (6, 69), (6, 3...","[(10, 138), (8, 125), (6, 127), (6, 69), (6, 3...",True


Compare os tempos médios de execução e uso de memória de cada algoritmo


In [93]:
queries_tempo_df = pd.DataFrame()
queries_tempo_df['tempo_médio_document_at_a_time'] = time_results_document
queries_tempo_df['tempo_médio_term_at_a_time'] = time_results_term
queries_tempo_df

,tempo_médio_document_at_a_time,tempo_médio_term_at_a_time
0,0.001993,0.0
1,0.000997,0.0
2,0.000997,0.0
3,0.000997,0.0
4,0.000997,0.0


3. Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).¶


In [94]:
def conj_query(query, inverted_index, k):
  
  query_indexes = [] 
  rank = [] 
  
  for word in query.split(" "):
    if word in inverted_index.keys():
        query_indexes.append(inverted_index[word])
  
  all_indexes = [item for sublist in query_indexes for item in sublist]
  all_indexes.sort() 
  
  for i in range(len(all_indexes)):
    doc_score = 0
    d = all_indexes.pop()
    repeat = 1
    for index in all_indexes:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

In [95]:
queries_conj = ["decisão sobre", "ministro paulo", "brasil país", "jair bolsonaro", "presidente disse"]


In [96]:
k = 10
results_conj = []

for query in queries_conj:
  score_conj = conj_query(query, inverted_index, k)
  results_conj.append(score_conj)

In [97]:
conj_queries_df = pd.DataFrame()
conj_queries_df['Query'] = queries_conj
conj_queries_df['Results'] = results_conj
conj_queries_df.index+=1
conj_queries_df

,Query,Results
1,decisão sobre,"[(12, 206), (10, 202), (10, 69), (10, 17), (7,..."
2,ministro paulo,"[(12, 240), (12, 209), (10, 215), (7, 151), (6..."
3,brasil país,"[(65, 151), (26, 166), (18, 173), (17, 19), (1..."
4,jair bolsonaro,"[(52, 151), (48, 207), (39, 166), (12, 216), (..."
5,presidente disse,"[(21, 63), (19, 166), (14, 19), (13, 235), (13..."
